In [2]:
import numpy as np
import random
from itertools import product
import pandas as pd
import sys
from joblib import Parallel, delayed

class Frota:
    def __init__(self, Periodo, TamanhoFrota,Demanda):
        while len(Demanda)<Periodo+1:
            Demanda.append(random.randint(1,TamanhoFrota))

        self.ProbQuebra_0 = 0.01
        self.Depreciacao = 0.4
        self.ProbPar = 0.4
        # self.Parametros = {}
        self.Periodo = Periodo
        self.TamanhoFrota = TamanhoFrota
        self.Demanda = Demanda
        self.ProbQuebraFcn = lambda x: 1-(1-self.ProbQuebra_0)*np.exp(-self.ProbPar*x)
        self.Estados = {}
        self.Acoes = {}
        # self.Acoes = {}
        self.Politica = []
        self.__num_estados__ = (Periodo+1)**TamanhoFrota
        self.__num_acoes__ = 2**TamanhoFrota
        self.ConstroeEstados2()
        self.ConstroeAcoes()
        
    
    def num2coefs(self,numero, base):
        coeficientes = [0]*self.TamanhoFrota
        if numero==0:
            return coeficientes
        
        cnt = 0
        while numero > 0:
            coeficiente = numero % base
            coeficientes[cnt]=coeficiente  # Insere o coeficiente à esquerda da lista
            numero //= base  # Divide o número pelo valor da base
            cnt = cnt+1
            
        return coeficientes

    def ConstroeAcoes(self):
        acoes = {}
        for i in range(self.Periodo+1):
            acoes[i]=[]
            for j in range(self.__num_acoes__):
                coefs = self.num2coefs(j,2)
                aux = self.TamanhoFrota-sum(coefs)
                if aux>=self.Demanda[i]:
                    acoes[i].append(tuple(coefs))
        
        self.Acoes = acoes 
    
    def ConstroeEstados(self):
        estados = {}
        for i in range(self.Periodo):
            for j in range(self.__num_estados__):
                coefs = self.num2coefs(j,self.Periodo+1)
                if max(coefs)<=i+1:
                    estados[(*coefs,i+1)]=0.5
        
        self.Estados = estados 


    def ConstroeEstados2(self):
        coefs = self.num2coefs(0,self.Periodo+1)
        estados = {}
        estados[(*coefs,0)] = 0.5
        estados[(*[-1]*self.TamanhoFrota,-1)] = 0
        for idx in range(self.__num_estados__):
            coefs = self.num2coefs(idx,self.Periodo+1)
            midx = max([*coefs,1])
            for j in range(midx,self.Periodo+1):
                estados[(*coefs,j)] = 0.5
        self.Estados = estados 

    def Recompensa(self,est):
        tvida = est[0]

        # define o valor do equipamento como função da idade do equipamento
        valor_equipamento = np.exp(-0.005*tvida)

        # define o valor da manutenção
        valor_manutencao = 0.02 - 0.01*np.exp(-0.1*tvida)
        
        # define o valor da manutenção se der defeito
        valor_defeito = -2*valor_manutencao/valor_equipamento
        
        # Cálculo das recompensas SEM defeito e COM defeito
        res_a0_op,res_a0_de = 0.01/valor_equipamento,valor_defeito

        # Cálculo das recompensas na manutenção
        res_a1 = -valor_manutencao/valor_equipamento
        
        # Cálculo das recompensas na troca de equipamento
        res_a2 = -(1 - 0.8*valor_equipamento)
        
        res = [res_a0_op,res_a0_de,res_a1,res_a2]
        if est[0]==-1:
            res = [0,0,0,0]
        return res

    def ProximoEstado(self,estado):
        acoes = self.EstadosAcoes[estado]
        NovosEstados = {}
        for ac in acoes:
            NovoEstado = [[]]*self.TamanhoFrota
            Recompensa = [[]]*self.TamanhoFrota
            Probabilidades = [[]]*self.TamanhoFrota
            for i,a in enumerate(ac):
                rcp_aux = self.Recompensa(estado[i])
                if a==0:
                    NovoEstado[i] =[(estado[i][0]+1,estado[i][1]+1),(estado[i][0]+1,0)]
                    prob_quebrar = self.ProbQuebraFcn(estado[i][1])
                    Probabilidades[i] = [1-prob_quebrar,prob_quebrar]
                    Recompensa[i] = [rcp_aux[0],rcp_aux[1]]
                if a==1:
                    NovoEstado[i] = [(estado[i][0]+1,0)]
                    Probabilidades[i] = [1]
                    Recompensa[i] = [rcp_aux[2]]
                if a==2:
                    NovoEstado[i] = [(0,0)]
                    Probabilidades[i] = [1]
                    Recompensa[i] = [rcp_aux[3]]
                if a==-1:
                    NovoEstado[i] = [(-1,-1)]
                    Probabilidades[i] = [1]
                    Recompensa[i] = [0]

            # print(NovoEstado)

            probs = list(product(*Probabilidades))
            recs = list(product(*Recompensa))
            probs = [np.prod(pr) for pr in probs]
            recs = [np.sum(rc) for rc in recs]
            NovosEstados["estados"]=[(*x,estado[-1]+1) for x in list(product(*NovoEstado))]
            NovosEstados["recompensas"]=recs
            NovosEstados["probabilidades"]=probs

        return NovosEstados

class Frota2:
    def __init__(self, Periodo, TamanhoFrota,Demanda):
        while len(Demanda)<Periodo+1:
            Demanda.append(random.randint(0,TamanhoFrota))

        self.ProbQuebra_0 = 0.01
        self.Depreciacao = 0.4
        self.ProbPar = 0.4
        self.Periodo = Periodo
        self.TamanhoFrota = TamanhoFrota
        self.Demanda = Demanda
        self.ProbQuebraFcn = lambda x: 1-(1-self.ProbQuebra_0)*np.exp(-self.ProbPar*x)
        self.Estados = {}
        self.Acoes = {}
        self.Politica = []
        self.__num_estados__ = (Periodo+1)**TamanhoFrota
        self.__num_acoes__ = 2**TamanhoFrota
        self.ConstroeEstadosP()
        self.ConstroeAcoes()
        
    
    def num2coefs(self,numero, base):
        coeficientes = [0]*self.TamanhoFrota
        if numero==0:
            return coeficientes
        
        cnt = 0
        while numero > 0:
            coeficiente = numero % base
            coeficientes[cnt]=coeficiente  # Insere o coeficiente à esquerda da lista
            numero //= base  # Divide o número pelo valor da base
            cnt = cnt+1
            
        return coeficientes

    def ConstroeAcoes(self):
        acoes = {}
        for i in range(self.Periodo+1):
            acoes[i]=[]
            for j in range(self.__num_acoes__):
                coefs = self.num2coefs(j,2)
                aux = self.TamanhoFrota-sum(coefs)
                if aux>=self.Demanda[i]:
                    acoes[i].append(j)
        
        self.Acoes = acoes 
    
    def ConstroeEstadosP(self):
        estados = {}
        estados[(0,0)] = 0.5
        estados[(-1,-1)] = 0
        tam = 100000

        def worker(start, end):
            estados_locais = {}
            for idx in range(start, end):
                coefs = self.num2coefs(idx, self.Periodo+1)
                midx = max([*coefs,1])
                for j in range(midx, self.Periodo+1):
                    estados_locais[(idx,j)] = 0.5
            
            return estados_locais
        
        intervals = [(i, min(i + tam, self.__num_estados__)) for i in range(0, self.__num_estados__, tam)]
        results = Parallel(n_jobs=-1)(delayed(worker)(start, end) for start, end in intervals)

        for result in results:
            estados.update(result)
        self.Estados = estados


    def ConstroeEstados3(self):
        estados = {}
        estados[(0,0)] = 0.5
        estados[(-1,-1)] = 0

        for idx in range(self.__num_estados__):
            coefs = self.num2coefs(idx,self.Periodo+1)
            midx = max([*coefs,1])
            for j in range(midx,self.Periodo+1):
                estados[(idx,j)] = 0.5
        self.Estados = estados 

    def Recompensa(self,v,p):
        tvida = 0

        # define o valor do equipamento como função da idade do equipamento
        valor_equipamento = np.exp(-0.005*tvida)

        # define o valor da manutenção
        valor_manutencao = 0.02/valor_equipamento
        
        # define o valor da manutenção se der defeito
        valor_defeito = 5*valor_manutencao
        
        # Cálculo das recompensas SEM defeito e COM defeito
        res_a0_op,res_a0_de = 0,-valor_defeito

        # Cálculo das recompensas na manutenção
        res_a1 = -valor_manutencao
        
        res = [res_a0_op,res_a0_de,res_a1]
        if p ==-1:
            res = [0,0,0]
        return res
    
    def coefs2num(self,coefs,base):
        num = 0
        for i,j in enumerate(coefs):
            num = num+j*base**i
        
        return num


    def ProximoEstado(self,estado):
        p = estado[-1]
        est = estado[0]
        acoes = self.Acoes[p]
        NovosEstados = {"estados":[],"recompensas":[], "probabilidades":[]}
        coefs = self.num2coefs(est,self.Periodo+1)
        if p in [self.Periodo,-1]:
            return {"estados":[(-1,-1)],"recompensas":[0], "probabilidades":[1]}
        for ac in acoes:
            ac_cf = self.num2coefs(ac,2)
            NovoEstado = [[]]*self.TamanhoFrota
            Recompensa = [[]]*self.TamanhoFrota
            Probabilidades = [[]]*self.TamanhoFrota
            for i,a in enumerate(ac_cf):
                rcp_aux = self.Recompensa(coefs[i],p)
                if a == 0:
                    NovoEstado[i] =[0,coefs[i]+1]
                    prob_quebrar = self.ProbQuebraFcn(coefs[i])
                    Probabilidades[i] = [prob_quebrar,1-prob_quebrar]
                    Recompensa[i] = [rcp_aux[0],rcp_aux[1]]
                if a == 1:
                    NovoEstado[i] = [0]
                    Probabilidades[i] = [1]
                    Recompensa[i] = [rcp_aux[2]]
                if a == -1:
                    NovoEstado[i] = [-1]
                    Probabilidades[i] = [1]
                    Recompensa[i] = [0]

            probs = list(product(*Probabilidades))
            recs = list(product(*Recompensa))
            probs = [np.prod(pr) for pr in probs]
            recs = [np.sum(rc) for rc in recs]
            NovosEstados["estados"].append([(self.coefs2num(x,2),estado[-1]+1) for x in list(product(*NovoEstado))])
            NovosEstados["recompensas"].append(recs)
            NovosEstados["probabilidades"].append(probs)

        return NovosEstados
    
    def mapa(self):
        



In [3]:
ft = Frota2(12,6,[1,2,1])

In [6]:
ft.Estados

{(0, 0): 0.5,
 (-1, -1): 0,
 (0, 1): 0.5,
 (0, 2): 0.5,
 (0, 3): 0.5,
 (0, 4): 0.5,
 (0, 5): 0.5,
 (0, 6): 0.5,
 (0, 7): 0.5,
 (0, 8): 0.5,
 (0, 9): 0.5,
 (0, 10): 0.5,
 (0, 11): 0.5,
 (0, 12): 0.5,
 (1, 1): 0.5,
 (1, 2): 0.5,
 (1, 3): 0.5,
 (1, 4): 0.5,
 (1, 5): 0.5,
 (1, 6): 0.5,
 (1, 7): 0.5,
 (1, 8): 0.5,
 (1, 9): 0.5,
 (1, 10): 0.5,
 (1, 11): 0.5,
 (1, 12): 0.5,
 (2, 2): 0.5,
 (2, 3): 0.5,
 (2, 4): 0.5,
 (2, 5): 0.5,
 (2, 6): 0.5,
 (2, 7): 0.5,
 (2, 8): 0.5,
 (2, 9): 0.5,
 (2, 10): 0.5,
 (2, 11): 0.5,
 (2, 12): 0.5,
 (3, 3): 0.5,
 (3, 4): 0.5,
 (3, 5): 0.5,
 (3, 6): 0.5,
 (3, 7): 0.5,
 (3, 8): 0.5,
 (3, 9): 0.5,
 (3, 10): 0.5,
 (3, 11): 0.5,
 (3, 12): 0.5,
 (4, 4): 0.5,
 (4, 5): 0.5,
 (4, 6): 0.5,
 (4, 7): 0.5,
 (4, 8): 0.5,
 (4, 9): 0.5,
 (4, 10): 0.5,
 (4, 11): 0.5,
 (4, 12): 0.5,
 (5, 5): 0.5,
 (5, 6): 0.5,
 (5, 7): 0.5,
 (5, 8): 0.5,
 (5, 9): 0.5,
 (5, 10): 0.5,
 (5, 11): 0.5,
 (5, 12): 0.5,
 (6, 6): 0.5,
 (6, 7): 0.5,
 (6, 8): 0.5,
 (6, 9): 0.5,
 (6, 10): 0.5,
 (6, 11): 0.5,


In [9]:
ft.num2coefs(194,7)

[5, 6, 3, 0, 0, 0]

In [8]:
ft.ProximoEstado((194,11))

{'estados': [[(0, 12),
   (32, 12),
   (16, 12),
   (48, 12),
   (8, 12),
   (40, 12),
   (24, 12),
   (56, 12),
   (8, 12),
   (40, 12),
   (24, 12),
   (56, 12),
   (16, 12),
   (48, 12),
   (32, 12),
   (64, 12),
   (4, 12),
   (36, 12),
   (20, 12),
   (52, 12),
   (12, 12),
   (44, 12),
   (28, 12),
   (60, 12),
   (12, 12),
   (44, 12),
   (28, 12),
   (60, 12),
   (20, 12),
   (52, 12),
   (36, 12),
   (68, 12),
   (13, 12),
   (45, 12),
   (29, 12),
   (61, 12),
   (21, 12),
   (53, 12),
   (37, 12),
   (69, 12),
   (21, 12),
   (53, 12),
   (37, 12),
   (69, 12),
   (29, 12),
   (61, 12),
   (45, 12),
   (77, 12),
   (17, 12),
   (49, 12),
   (33, 12),
   (65, 12),
   (25, 12),
   (57, 12),
   (41, 12),
   (73, 12),
   (25, 12),
   (57, 12),
   (41, 12),
   (73, 12),
   (33, 12),
   (65, 12),
   (49, 12),
   (81, 12)],
  [(0, 12),
   (32, 12),
   (16, 12),
   (48, 12),
   (8, 12),
   (40, 12),
   (24, 12),
   (56, 12),
   (8, 12),
   (40, 12),
   (24, 12),
   (56, 12),
   (16,

In [10]:
res = ft.ProximoEstado((194,11))["estados"][0]

In [16]:
res

[(0, 12),
 (32, 12),
 (16, 12),
 (48, 12),
 (8, 12),
 (40, 12),
 (24, 12),
 (56, 12),
 (8, 12),
 (40, 12),
 (24, 12),
 (56, 12),
 (16, 12),
 (48, 12),
 (32, 12),
 (64, 12),
 (4, 12),
 (36, 12),
 (20, 12),
 (52, 12),
 (12, 12),
 (44, 12),
 (28, 12),
 (60, 12),
 (12, 12),
 (44, 12),
 (28, 12),
 (60, 12),
 (20, 12),
 (52, 12),
 (36, 12),
 (68, 12),
 (13, 12),
 (45, 12),
 (29, 12),
 (61, 12),
 (21, 12),
 (53, 12),
 (37, 12),
 (69, 12),
 (21, 12),
 (53, 12),
 (37, 12),
 (69, 12),
 (29, 12),
 (61, 12),
 (45, 12),
 (77, 12),
 (17, 12),
 (49, 12),
 (33, 12),
 (65, 12),
 (25, 12),
 (57, 12),
 (41, 12),
 (73, 12),
 (25, 12),
 (57, 12),
 (41, 12),
 (73, 12),
 (33, 12),
 (65, 12),
 (49, 12),
 (81, 12)]

In [27]:
x = [v[0] for v in res]
y =  [v[1] for v in res]

In [28]:
x

[0,
 32,
 16,
 48,
 8,
 40,
 24,
 56,
 8,
 40,
 24,
 56,
 16,
 48,
 32,
 64,
 4,
 36,
 20,
 52,
 12,
 44,
 28,
 60,
 12,
 44,
 28,
 60,
 20,
 52,
 36,
 68,
 13,
 45,
 29,
 61,
 21,
 53,
 37,
 69,
 21,
 53,
 37,
 69,
 29,
 61,
 45,
 77,
 17,
 49,
 33,
 65,
 25,
 57,
 41,
 73,
 25,
 57,
 41,
 73,
 33,
 65,
 49,
 81]